In [1]:
import numpy as np
import pandas as pd
import mne
import csv, json
import datetime

In [2]:
sample_folder = './samples/'
file_name = 'Veh70_02202015'

In [3]:
def edf_to_dataset(edf_path, scores_path):
    edf = mne.io.read_raw_edf(edf_path)
    sampling_rate = int(edf.info['sfreq'])
    name = datetime.datetime.utcfromtimestamp(edf.info['meas_date'][0]).strftime('%Y-%m-%d %H:%M:%S')
    
    header = (','.join(edf.ch_names)).split(',')
    df = edf.get_data().T
    
    scores = np.squeeze( pd.read_excel(scores_path, keep_default_na=False).values )
    values_per_window = int(df.shape[0]/scores.shape[0])
    num_windows = scores.shape[0]
    
    df = df[:num_windows*values_per_window]
    df = np.reshape(df, newshape=(num_windows, values_per_window, 2))

    scores[scores=='D'] = 'WA'
    
    return df, scores

In [28]:
def labels_to_onehot(Y, categories = {}):
    Y = Y.copy()
    count = 0
    for i in range(len(Y)):
        value = Y[i]
        if not value in categories:
            categories[value] = count
            count += 1
        Y[i] = categories[value]

    Y = Y.astype(int)
    Y_onehot = np.zeros((Y.shape[0], len(categories)))
#     print(Y_onehot.shape)
#     print(categories)
    Y_onehot[np.arange(Y.shape[0]), Y] = 1

    return Y_onehot, categories

In [29]:
X, Y = edf_to_dataset(sample_folder+file_name+'.edf', sample_folder+file_name+'.xls')
print(X.shape, Y.shape)

Extracting EDF parameters from C:\Users\Albert\Documents\AutoScorer\model\samples\Veh70_02202015.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
(2160, 5000, 2) (2160,)


In [35]:
Y_onehot, categories = labels_to_onehot(Y)
print(Y_onehot.shape)
print(categories)

(2160, 6)
{'NA': 0, 'WA': 1, 'W': 2, 'NR': 3, 'RA': 4, 'R': 5}
